# Exploring Periodicity

## Motivation

After comparing time series plots of various aspects of the data, the total unit sales stuck out as particularly interesting due to what appeared to be a stable periodicity. We judged that if this periodic trend of the total unit sales could be exploited and accurately predicted, then we could just focus on predicting the proportions of sales per store-item pair, effectively standardizing the scale of inputs and expected outputs of any subsequent models.

Here was the motivating plot of total unit sales over the last year, in which you can see a striking consistency in the spacing between spikes. 
![Total Last Year](periodicity/total_over_last_year.png)

## Hypothesis and Assumptions

Since the metric for this competition is the Root-Mean-Squared-Error of the *log* of predictions and truth, we felt it a natural choice to divide our prediction strategy into separate magnitude and proportion components. In doing so, we recognize the greater penalty for wrong-magnitude predictions over same-magnitude errors.

Given that there are 54 individual stores and 4100 items, we started with the assumption that no single store-item could greatly influence the total unit sales. We hypothesized it would be easier to determine the magnitude of our predictions from the total unit sales than from store-item specific training because, as the coarsest view of the data, the total unit sales would likely be the least noisy and suffer less from data sparsity. We planned to refine our view of the data successively (i.e. predict per-store unit sales next) if needed to predict magnitudes at a more detailed scale if needed.

## Exploration

Taking the Discrete Fourier Transform of the data we saw that there were three clear peaks but also some incoherent energy at lower frequencies. (The x-axis is labelled in units of "days per cycle" instead of the typical units of "Hz" to allow for easier interpretation in the context of the problem.)

![DFT of Original Data Over All Years](periodicity/original_fft_all_years.png)

We suspected the results could be made more coherent if the signal was decomposed by band-pass filters into a filter bank (i.e. hierarchy of frequencies). While wavelets are a typical choice for such purposes, we chose to use a simpler and perhaps more intuitive approach via the Laplacian pyramid transformation, which is a technique borrowed from the image processing community.

### Laplacian Pyramid

The Laplacian pyramid is a transformation that allows perfect reconstruction and separates the edges of an image at different scales. It successively convolves an image with a Gaussian kernel (blurring the image) and subtracts the convolved image from the original, leaving predominantly the highest frequencies, since the Gaussian kernel is a low pass filter. As each successive "level" of the pyramid is generated by this process, the subtraction is essentially taken between two signals filtered with Gaussian kernels of different scales (standard deviations). The original image is the result of previously performed Gaussian blurs, while the next image is filtered by one additional Gaussian blur. The scales of the Gaussians are effectively different (even though the kernel is unchanged throughout the process) because the signal is successively being downsampled. It so happens that the difference of two Gaussian filters with scale factor ~1.6 closely approximates the "Laplacian of Gaussians" filter commonly used in image processing for edge detection, hence the origin of the name "Laplacian pyramid."

Here is an example of the Laplacian pyramid performed on an image for demonstration. The results have been scaled to the same size but are actually downsampled by a factor of 2 horizontally and vertically at each stage of the pyramid. The colored image at the end is the highest level of the pyramid (the images are shown in ascending order of pyramid levels). The highest level contains the lowest frequency content.

<img src="periodicity/lpyr_example.png" title="Laplacian Pyramid example on an image" height="300" width="120">

The implementation of the Laplacian pyramid is simple as it only involves convolution with a Gaussian kernel and downsampling. The reconstruction process is also straightforward. It occurs by blurring and upsampling the highest levels of the pyramid and adding the result to the previous level until all levels have been collapsed back to the original. We were able to implement this transformation in all but 70 lines of Python code using numpy to handle the convolution. (Code can be found in our repository at [pyramid.py](periodicity/pyramid.py).)

Here is the result of the DFT on total unit sales separated into different levels of the Laplacian pyramid. The lowest levels of the pyramid intuitively should contain more of the higher component frequencies of the signal. (The y-axis of these plots is in transformed units, not the original total unit sales. We removed the linear and loglinear trends of the data to provide better conditions for the DFT.)

![Level 0 All Years](periodicity/level0_all_years.png)
![Level 1 and 2 All Years](periodicity/level1_2_all_years.png)
![Level 3 and 4 All Years](periodicity/level3_4_all_years.png)
![Level 5 and 6 All Years](periodicity/level5_6_all_years.png)
![Level 7 All Years](periodicity/level7_all_years.png)
![Level 8 All Years](periodicity/level8_all_years.png)
![Level 9 All Years](periodicity/level9_all_years.png)

While the spectra become more uniform (incoherent) as we progress to higher levels of the pyramid (corresponding to lower frequencies of the original signal), the magnitude of the coefficients is also decreasing significantly. Here is the same information but in a stacked plot to show the coefficients of different levels in the same y-axis scale.

![DFT of Laplacian Pyramid Transformed Data Over All Years](periodicity/lpyr_fft_all_years.png)

We can hardly see the influence of higher pyramid levels and we notice the incoherency at lower frequencies is gone in this stacked view. This tells us the highest frequency "edges" of our signal (the spikes) indeed happen at 2.3, 3.5, and 7 day cycles. Indeed we can see this periodicity manifest well in a plot of the last 60 days of the data, plotted with pyramid levels separated.

![Total Unit Sales of Laplacian Pyramid Transformed Data Over Last 60 Days](periodicity/lpyr_total_over_60_days.png)

Compare to without the pyramid transformation over the same period of time.

![Total Unit Sales of Data Over Last 60 Days](periodicity/total_over_60_days.png)

While qualitatively very similar, the Fourier spectra indicate that by separating the signal into pyramid levels we can recover a more coherent periodic pattern of the signal. We hoped to predict total unit sales using just the dominant signals of each pyramid level, but found mixed results.

Here are plots of thresholding the DFT coefficients.

![Thresholded DFT](periodicity/threshold_lpyr.png)

Here is the recovered threshold and pyramid reconstructed signal versus the original (over last 15 days of each yearly segment in the data).

<img src="periodicity/threshold_vs_original_15_days.png" width=450 height=450 title="Thresholded DFT Recovered vs. Original">

Here are the corresponding residuals.

![Thresholded DFT Recovered Residuals (15 Days of each year)](periodicity/threshold_residuals_15_days.png)

We hoped to use these periodicity-based insights as inputs to our models.